In [1]:
import os
from pprint import pprint

from langchain import hub
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

model = "nomic-embed-text-v1.5"
url = "http://10.147.20.161:12345/v1/"

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [27]:
# noinspection PyArgumentList
# llm = ChatOpenAI(api_key="hovno", base_url=url, model="llama-3.2-1b-instruct")
llm = ChatOpenAI(api_key="hovno", base_url=url, model="qwen2.5-14b-instruct")
# llm = OllamaLLM(model="llama3.2:3b")
embedding_llm = ChatOpenAI(api_key="hovno", base_url=url, model=model)

In [28]:
llm.invoke([HumanMessage(content="Hi. I am Bob.")]).content

"Hello Bob! It's nice to meet you. How can I assist you today?"

In [4]:
irc5_project_paths = ["../shared/zdrojaky/IRC5/" + filename + "/RAPID/" for filename in
                      os.listdir("../shared/zdrojaky/IRC5/")]
omnicore_project_paths = ["../shared/zdrojaky/Omnicore/" + filename + "/RAPID/" for filename in
                          os.listdir("../shared/zdrojaky/Omnicore/")]
pdf_paths = ['../shared/zdrojaky/Rapid tech manuals/' + filename for filename in
             os.listdir("../shared/zdrojaky/Rapid tech manuals")
             if filename.endswith(".pdf")]

In [5]:
# all_file_paths = [all_file_paths[4]]

pdf_documents = []
for index, file_path in enumerate(pdf_paths):
    loader = PyPDFLoader(file_path=file_path)
    document = loader.load()
    for page in document:
        page.metadata["version_number"] = 7 if "7" in file_path else 5
        page.metadata["version_name"] = "Omnicore" if "7" in file_path else "IRC5"
    pdf_documents.extend(document)

In [6]:
source_documents = []

for index, file_path in enumerate(irc5_project_paths):
    loader = DirectoryLoader(path=file_path, recursive=True, show_progress=True, glob="**/*.mod")
    document = loader.load()
    for page in document:
        page.metadata["version_number"] = 5
        page.metadata["version_name"] = "IRC5"
    source_documents.extend(document)
    print("Document", index, "loaded.")
    if index > 1:
        break

for index, file_path in enumerate(omnicore_project_paths):
    # ignore files ending with .sys extension
    loader = DirectoryLoader(path=file_path, recursive=True, show_progress=True, glob="**/*.modx", silent_errors=True)
    document = loader.load()
    for page in document:
        page.metadata["version_number"] = 7
        page.metadata["version_name"] = "Omnicore"
    source_documents.extend(document)
    print("Document", index, "loaded.")
    if index > 1:
        break

100%|██████████| 2/2 [00:00<00:00,  2.43it/s]


Document 0 loaded.


100%|██████████| 13/13 [00:01<00:00,  7.11it/s]


Document 1 loaded.


100%|██████████| 1/1 [00:00<00:00, 35.29it/s]


...

Document 0 loaded.


0it [00:00, ?it/s]


Document 1 loaded.


100%|██████████| 1/1 [00:00<00:00, 542.39it/s]

Document 2 loaded.


In [7]:
print("PDF documents before splitting:", len(pdf_documents))
print("Source documents before splitting:", len(source_documents))

PDF documents before splitting: 5940
Source documents before splitting: 22


In [8]:

# embeddings = OpenAIEmbeddings(model="e5-mistral-7b-instruct", api_key="hovno", base_url="http://10.147.20.161:12345/v1/",
#                           check_embedding_ctx_length=False)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=1000)
split_documents = text_splitter.split_documents(pdf_documents + source_documents)
# split_documents = text_splitter.split_documents(other_do)
print(len(split_documents))

pprint(split_documents[:10])
# print("Document", index, "loaded.")
# embeddings = OpenAIEmbeddings(model="nomic-embed-text-v1.5", api_key="hovno", base_url="http://10.147.20.161:12345/v1/",
#                               )


6103
[Document(metadata={'source': '../shared/zdrojaky/Rapid tech manuals/TRM RAPID RW 5-en.pdf', 'page': 0, 'version_number': 5, 'version_name': 'IRC5'}, page_content='Technical reference manual\nRAPIDInstructions, Functions andData\ntypes'),
 Document(metadata={'source': '../shared/zdrojaky/Rapid tech manuals/TRM RAPID RW 5-en.pdf', 'page': 1, 'version_number': 5, 'version_name': 'IRC5'}, page_content='Tracebackinformation:\nWorkspace R14-1versiona5\nCheckedin2014-04-07\nSkribenta version4.0.378'),
 Document(metadata={'source': '../shared/zdrojaky/Rapid tech manuals/TRM RAPID RW 5-en.pdf', 'page': 2, 'version_number': 5, 'version_name': 'IRC5'}, page_content='Technical reference manual\nRAPIDInstructions, Functions andDatatypes\nRobotWare 5.61\nDocument ID:3HAC16581-1\nRevision: R\n©Copyright 2004-2014ABB.Allrightsreserved.'),
 Document(metadata={'source': '../shared/zdrojaky/Rapid tech manuals/TRM RAPID RW 5-en.pdf', 'page': 3, 'version_number': 5, 'version_name': 'IRC5'}, page_cont

In [9]:
embeddings = OpenAIEmbeddings(model=model, api_key="hovno", base_url=url, check_embedding_ctx_length=False)
vectorstore = Chroma.from_documents(documents=split_documents, embedding=embeddings)

In [36]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(k=2)
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)

add_documentation = True
if add_documentation:
    doc_string = "Also add documentation to the code."
else:
    doc_string = ""
# rewrite this /|\ using older Python syntax

db = Chroma.from_documents(split_documents, embedding_function=embeddings, persist_directory="./chroma_db")
db.persist()

TypeError: langchain_chroma.vectorstores.Chroma() got multiple values for keyword argument 'embedding_function'

In [30]:
# rag_chain.invoke("Do you know what's Rapid RW 5?")
output = rag_chain.invoke(f"""You are a machine that translates version 5 (IRC5) code into newer version 7 (Omnicore). You don't add any other words to the response, you only output the resulting code. Here's the code:
```PROC close_gripper(num i)
        err:=1;
        WHILE err <> 0 DO
            TEST i
                CASE 1:
                    err:= gripper_bottom_close();
                CASE 2:
                    err:= gripper_top_close();
                CASE 3:
                    err:= gripper_band_close();
                CASE 4:
                    err:= make_vacuum();
            ENDTEST
            !check the value of the sensors
            IF err <> 0 THEN 
                SetDO rbt_anomaly, 1;
                check_error_close(err);
            ELSE SetDO rbt_anomaly, 0;        
            ENDIF  
        ENDWHILE
    ENDPROC
```
This is the end of the input code. Don't output ANYTHING other than the output code surrounded by triple backticks.
{doc_string}
""")

with open("output.txt", "w") as f:
    f.write(output)